Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 7.53 ms, sys: 0 ns, total: 7.53 ms
Wall time: 12.9 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

<Client: 'inproc://10.1.1.131/7919/1' processes=1 threads=6, memory=6.78 GiB>

1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 634 ms, sys: 40.6 ms, total: 675 ms
Wall time: 2.12 s


[1.1386945694539632,
 1.8672538937999086,
 1.7750175921725815,
 1.8781237123158516,
 1.6360443672039522,
 1.0323268556077894,
 1.2829855354130764,
 1.122880768294328,
 1.8792582030637979,
 1.8309115314475337,
 1.4605481681306998,
 1.9397709154056642,
 1.9742027096744272,
 1.3458234823138777,
 1.564053837474913,
 1.5161476234075653,
 1.4136176192975718,
 1.9076710935976982,
 1.6393149614442268,
 1.8227134093418211,
 1.966979122520502,
 1.4820238918221451,
 1.7659175555223556,
 1.124450696595173,
 1.766477924519682,
 1.4443892279650712,
 1.8680751914842142,
 1.3255162878299602,
 1.8252310063659607,
 1.4633616195038501,
 1.9150356378806634,
 1.4898953305010534,
 1.0607024280655821,
 1.2790021577929458,
 1.7937112897417915,
 1.7520877768820111,
 1.1188295120061622,
 1.4485503445653334]

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.139875389530763,
 1.0792500872539028,
 1.8897097120071786,
 1.3872113900987155,
 1.900074184160546,
 1.5639929402323083,
 1.64049187566035,
 1.7432148169001347,
 1.5786005840491255,
 1.458780807424581,
 1.42378703264491,
 1.5533442216457098,
 1.4679403418676078,
 1.8500209573076891,
 1.6949039676921076,
 1.494785412082337,
 1.0775339869965719,
 1.3136754836673383,
 1.6922955876511974,
 1.2720247408447645,
 1.769086468554324,
 1.1309741986262098,
 1.1585855708715238,
 1.8552480677400547,
 1.9329881337239931,
 1.7536705058639386,
 1.7428959463070877,
 1.9147341842203018,
 1.4091880658873763,
 1.535030940305753,
 1.2106578936950498,
 1.68715385763252,
 1.561650831425696,
 1.3533796051850149,
 1.5695350060224669,
 1.093739131984769,
 1.031206987010568,
 1.9965918548523742,
 1.4857632968135142,
 1.5302442121878255,
 1.3118350945181936,
 1.529575125471499,
 1.5224542672312031,
 1.4671785145557101,
 1.4797004680792907,
 1.1624095511762529,
 1.032546210954575,
 1.563936404498486,
 1.4653418

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 344 ms, sys: 30.7 ms, total: 375 ms
Wall time: 2.8 s
